# Training Models at Scale with AI Platform
**Learning Objectives:**
  1. Learn how to organize your training code into a Python package
  1. Train your model using cloud infrastructure via Google Cloud AI Platform Training Service
  1. (optional) Learn how to run your training package using Docker containers and push training Docker images on a Docker registry

## Introduction

In this notebook we'll make the jump from training locally, to do training in the cloud. We'll take advantage of Google Cloud's [AI Platform Training Service](https://cloud.google.com/ai-platform/). 

AI Platform Training Service is a managed service that allows the training and deployment of ML models without having to provision or maintain servers. The infrastructure is handled seamlessly by the managed service for us.

Each learning objective will correspond to a __#TODO__ in the [student lab notebook](https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/art_and_science_of_ml/labs/training_models_at_scale.ipynb) -- try to complete that notebook first before reviewing this solution notebook.

In [1]:
# Use the chown command to change the ownership of repository to user
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [2]:
# Install the Google Cloud BigQuery
!pip install --user google-cloud-bigquery==1.25.0

**Note**: Restart your kernel to use updated packages.

Kindly ignore the deprecation warnings and incompatibility errors related to google-cloud-storage.

Specify your project name, bucket name and region in the cell below.

In [3]:
# The OS module in Python provides functions for interacting with the operating system
import os

from google.cloud import bigquery

In [4]:
# Change with your own bucket and project below:
BUCKET =  "qwiklabs-gcp-01-40af11de1e73"
PROJECT = "qwiklabs-gcp-01-40af11de1e73"
REGION = "us-east1"

OUTDIR = "gs://{bucket}/taxifare/data".format(bucket=BUCKET)

# Store the value of `BUCKET`, `OUTDIR`, `PROJECT`, `REGION` and `TFVERSION` in environment variables.
os.environ['BUCKET'] = BUCKET
os.environ['OUTDIR'] = OUTDIR
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = "2.1"

In [5]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


## Create BigQuery tables

If you have not already created a BigQuery dataset for our data, run the following cell:

In [6]:
# Created a BigQuery dataset for our data
bq = bigquery.Client(project = PROJECT)
dataset = bigquery.Dataset(bq.dataset("taxifare"))

try:
    bq.create_dataset(dataset)
    print("Dataset created")
except:
    print("Dataset already exists")

Dataset already exists


Let's create a table with 1 million examples.

Note that the order of columns is exactly what was in our CSV files.

In [7]:
%%bigquery

# Creating the table in our dataset.
CREATE OR REPLACE TABLE taxifare.feateng_training_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `nyc-tlc.yellow.trips`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 1000)) = 1
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

""


Make the validation dataset be 1/10 the size of the training dataset.

In [8]:
%%bigquery

# Creating the table in our dataset.
CREATE OR REPLACE TABLE taxifare.feateng_valid_data AS

SELECT
    (tolls_amount + fare_amount) AS fare_amount,
    pickup_datetime,
    pickup_longitude AS pickuplon,
    pickup_latitude AS pickuplat,
    dropoff_longitude AS dropofflon,
    dropoff_latitude AS dropofflat,
    passenger_count*1.0 AS passengers,
    'unused' AS key
FROM `nyc-tlc.yellow.trips`
WHERE ABS(MOD(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING)), 10000)) = 2
AND
    trip_distance > 0
    AND fare_amount >= 2.5
    AND pickup_longitude > -78
    AND pickup_longitude < -70
    AND dropoff_longitude > -78
    AND dropoff_longitude < -70
    AND pickup_latitude > 37
    AND pickup_latitude < 45
    AND dropoff_latitude > 37
    AND dropoff_latitude < 45
    AND passenger_count > 0

""


## Export the tables as CSV files

In [9]:
%%bash

# Deleting the current contents of output directory.
echo "Deleting current contents of $OUTDIR"
gsutil -m -q rm -rf $OUTDIR

# Fetching the training data to output directory.
echo "Extracting training data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_training_data \
   $OUTDIR/taxi-train-*.csv

echo "Extracting validation data to $OUTDIR"
bq --location=US extract \
   --destination_format CSV  \
   --field_delimiter "," --noprint_header \
   taxifare.feateng_valid_data \
   $OUTDIR/taxi-valid-*.csv

# The `ls` command will show the content of working directory
gsutil ls -l $OUTDIR

Deleting current contents of gs://qwiklabs-gcp-01-40af11de1e73/taxifare/data
Extracting training data to gs://qwiklabs-gcp-01-40af11de1e73/taxifare/data
Extracting validation data to gs://qwiklabs-gcp-01-40af11de1e73/taxifare/data
  88345235  2021-01-27T17:53:47Z  gs://qwiklabs-gcp-01-40af11de1e73/taxifare/data/taxi-train-000000000000.csv
   8725746  2021-01-27T17:53:54Z  gs://qwiklabs-gcp-01-40af11de1e73/taxifare/data/taxi-valid-000000000000.csv
TOTAL: 2 objects, 97070981 bytes (92.57 MiB)


Waiting on bqjob_r3f627347e1226ea5_0000017744fa8be6_1 ... (23s) Current status: DONE   
Waiting on bqjob_r32c6a2a4aabb27d6_0000017744faeeb4_1 ... (2s) Current status: DONE   


In [10]:
# The `cat` command will outputs the contents of one or more URLs
# Using `head -2` we are showing only top two output files
!gsutil cat gs://$BUCKET/taxifare/data/taxi-train-000000000000.csv | head -2

33.870000000000005,2010-05-16 13:11:25 UTC,-73.864352,40.770916,-73.995966,40.749129,2,unused
17,2014-07-12 21:09:34 UTC,-74.001758,40.722579,-73.957957,40.717557,2,unused


## Make code compatible with AI Platform Training Service
In order to make our code compatible with AI Platform Training Service we need to make the following changes:

1. Upload data to Google Cloud Storage 
2. Move code into a trainer Python package
4. Submit training job with `gcloud` to train on AI Platform

### Upload data to Google Cloud Storage (GCS)

Cloud services don't have access to our local files, so we need to upload them to a location the Cloud servers can read from. In this case we'll use GCS.

In [11]:
# The `ls` command will show the content of working directory
!gsutil ls gs://$BUCKET/taxifare/data

gs://qwiklabs-gcp-01-40af11de1e73/taxifare/data/taxi-train-000000000000.csv
gs://qwiklabs-gcp-01-40af11de1e73/taxifare/data/taxi-valid-000000000000.csv


### Move code into a Python package


The first thing to do is to convert your training code snippets into a regular Python package that we will then `pip install` into the Docker container. 

A Python package is simply a collection of one or more `.py` files along with an `__init__.py` file to identify the containing directory as a package. The `__init__.py` sometimes contains initialization code but for our purposes an empty file suffices.

#### Create the package directory

Our package directory contains 3 files:

In [12]:
ls ./taxifare/trainer/

__init__.py  __pycache__/  model.py  task.py


#### Paste existing code into model.py

A Python package requires our code to be in a .py file, as opposed to notebook cells. So, we simply copy and paste our existing code for the previous notebook into a single file.

In the cell below, we write the contents of the cell into `model.py` packaging the model we 
developed in the previous labs so that we can deploy it to AI Platform Training Service.  

In [13]:
%%writefile ./taxifare/trainer/model.py
# The datetime module used to work with dates as date objects.
import datetime
# The logging module in Python allows writing status messages to a file or any other output streams. 
import logging
# The OS module in Python provides functions for interacting with the operating system
import os
# The shutil module in Python provides many functions of high-level operations on files and collections of files.
# This module helps in automating process of copying and removal of files and directories.
import shutil

# Here we'll import data processing libraries like Numpy and Tensorflow
import numpy as np
import tensorflow as tf

from tensorflow.keras import activations
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras import models

from tensorflow import feature_column as fc

logging.info(tf.version.VERSION)


# Defining the feature names into a list `CSV_COLUMNS`
CSV_COLUMNS = [
        'fare_amount',
        'pickup_datetime',
        'pickup_longitude',
        'pickup_latitude',
        'dropoff_longitude',
        'dropoff_latitude',
        'passenger_count',
        'key',
]
LABEL_COLUMN = 'fare_amount'
# Defining the default values into a list `DEFAULTS`
DEFAULTS = [[0.0], ['na'], [0.0], [0.0], [0.0], [0.0], [0.0], ['na']]
DAYS = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']


def features_and_labels(row_data):
    for unwanted_col in ['key']:
        row_data.pop(unwanted_col)
# The .pop() method will return item and drop from frame.
    label = row_data.pop(LABEL_COLUMN)
    return row_data, label


def load_dataset(pattern, batch_size, num_repeat):
# The tf.data.experimental.make_csv_dataset() method reads CSV files into a dataset
    dataset = tf.data.experimental.make_csv_dataset(
        file_pattern=pattern,
        batch_size=batch_size,
        column_names=CSV_COLUMNS,
        column_defaults=DEFAULTS,
        num_epochs=num_repeat,
    )
# The `map()` function executes a specified function for each item in an iterable.
# The item is sent to the function as a parameter.
    return dataset.map(features_and_labels)


def create_train_dataset(pattern, batch_size):
    dataset = load_dataset(pattern, batch_size, num_repeat=None)
# The `prefetch()` method will start a background thread to populate a ordered buffer that acts like a queue, so that downstream pipeline stages need not block.
    return dataset.prefetch(1)


def create_eval_dataset(pattern, batch_size):
    dataset = load_dataset(pattern, batch_size, num_repeat=1)
# The `prefetch()` method will start a background thread to populate a ordered buffer that acts like a queue, so that downstream pipeline stages need not block.
    return dataset.prefetch(1)


def parse_datetime(s):
    if type(s) is not str:
        s = s.numpy().decode('utf-8')
    return datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S %Z")


def euclidean(params):
    lon1, lat1, lon2, lat2 = params
    londiff = lon2 - lon1
    latdiff = lat2 - lat1
    return tf.sqrt(londiff*londiff + latdiff*latdiff)


def get_dayofweek(s):
    ts = parse_datetime(s)
    return DAYS[ts.weekday()]


@tf.function
def dayofweek(ts_in):
    return tf.map_fn(
        lambda s: tf.py_function(get_dayofweek, inp=[s], Tout=tf.string),
        ts_in
    )


@tf.function
def fare_thresh(x):
    return 60 * activations.relu(x)


def transform(inputs, NUMERIC_COLS, STRING_COLS, nbuckets):
    # Pass-through columns
    transformed = inputs.copy()
    del transformed['pickup_datetime']

    feature_columns = {
        colname: fc.numeric_column(colname)
        for colname in NUMERIC_COLS
    }

    # Scaling longitude from range [-70, -78] to [0, 1]
    for lon_col in ['pickup_longitude', 'dropoff_longitude']:
        transformed[lon_col] = layers.Lambda(
            lambda x: (x + 78)/8.0,
            name='scale_{}'.format(lon_col)
        )(inputs[lon_col])

    # Scaling latitude from range [37, 45] to [0, 1]
    for lat_col in ['pickup_latitude', 'dropoff_latitude']:
        transformed[lat_col] = layers.Lambda(
            lambda x: (x - 37)/8.0,
            name='scale_{}'.format(lat_col)
        )(inputs[lat_col])

    # Adding Euclidean dist (no need to be accurate: NN will calibrate it)
    transformed['euclidean'] = layers.Lambda(euclidean, name='euclidean')([
        inputs['pickup_longitude'],
        inputs['pickup_latitude'],
        inputs['dropoff_longitude'],
        inputs['dropoff_latitude']
    ])
    feature_columns['euclidean'] = fc.numeric_column('euclidean')

    # hour of day from timestamp of form '2010-02-08 09:17:00+00:00'
    transformed['hourofday'] = layers.Lambda(
        lambda x: tf.strings.to_number(
            tf.strings.substr(x, 11, 2), out_type=tf.dtypes.int32),
        name='hourofday'
    )(inputs['pickup_datetime'])
    feature_columns['hourofday'] = fc.indicator_column(
        fc.categorical_column_with_identity(
            'hourofday', num_buckets=24))

    latbuckets = np.linspace(0, 1, nbuckets).tolist()
    lonbuckets = np.linspace(0, 1, nbuckets).tolist()
    b_plat = fc.bucketized_column(
        feature_columns['pickup_latitude'], latbuckets)
    b_dlat = fc.bucketized_column(
            feature_columns['dropoff_latitude'], latbuckets)
    b_plon = fc.bucketized_column(
            feature_columns['pickup_longitude'], lonbuckets)
    b_dlon = fc.bucketized_column(
            feature_columns['dropoff_longitude'], lonbuckets)
    ploc = fc.crossed_column(
            [b_plat, b_plon], nbuckets * nbuckets)
    dloc = fc.crossed_column(
            [b_dlat, b_dlon], nbuckets * nbuckets)
    pd_pair = fc.crossed_column([ploc, dloc], nbuckets ** 4)
    feature_columns['pickup_and_dropoff'] = fc.embedding_column(
            pd_pair, 100)

    return transformed, feature_columns


def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))


def build_dnn_model(nbuckets, nnsize, lr):
    # input layer is all float except for pickup_datetime which is a string
    STRING_COLS = ['pickup_datetime']
    NUMERIC_COLS = (
            set(CSV_COLUMNS) - set([LABEL_COLUMN, 'key']) - set(STRING_COLS)
    )
    inputs = {
        colname: layers.Input(name=colname, shape=(), dtype='float32')
        for colname in NUMERIC_COLS
    }
    inputs.update({
        colname: layers.Input(name=colname, shape=(), dtype='string')
        for colname in STRING_COLS
    })

    # transforms
    transformed, feature_columns = transform(
        inputs, NUMERIC_COLS, STRING_COLS, nbuckets=nbuckets)
    dnn_inputs = layers.DenseFeatures(feature_columns.values())(transformed)

    x = dnn_inputs
    for layer, nodes in enumerate(nnsize):
        x = layers.Dense(nodes, activation='relu', name='h{}'.format(layer))(x)
    output = layers.Dense(1, name='fare')(x)

    model = models.Model(inputs, output)
    #TODO 1a
    lr_optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(optimizer=lr_optimizer, loss='mse', metrics=[rmse, 'mse'])
    
    return model


def train_and_evaluate(hparams):
    #TODO 1b
    batch_size = hparams['batch_size'] 
    nbuckets = hparams['nbuckets'] 
    lr = hparams['lr'] 
    nnsize = hparams['nnsize']
    eval_data_path = hparams['eval_data_path']
    num_evals = hparams['num_evals']
    num_examples_to_train_on = hparams['num_examples_to_train_on']
    output_dir = hparams['output_dir']
    train_data_path = hparams['train_data_path']

    timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    savedmodel_dir = os.path.join(output_dir, 'export/savedmodel')
    model_export_path = os.path.join(savedmodel_dir, timestamp)
    checkpoint_path = os.path.join(output_dir, 'checkpoints')
    tensorboard_path = os.path.join(output_dir, 'tensorboard')

    if tf.io.gfile.exists(output_dir):
        tf.io.gfile.rmtree(output_dir)

    model = build_dnn_model(nbuckets, nnsize, lr)
    logging.info(model.summary())

    trainds = create_train_dataset(train_data_path, batch_size)
    evalds = create_eval_dataset(eval_data_path, batch_size)

    steps_per_epoch = num_examples_to_train_on // (batch_size * num_evals)

    checkpoint_cb = callbacks.ModelCheckpoint(
        checkpoint_path,
        save_weights_only=True,
        verbose=1
    )
    tensorboard_cb = callbacks.TensorBoard(tensorboard_path)

    history = model.fit(
        trainds,
        validation_data=evalds,
        epochs=num_evals,
        steps_per_epoch=max(1, steps_per_epoch),
        verbose=2,  # 0=silent, 1=progress bar, 2=one line per epoch
        callbacks=[checkpoint_cb, tensorboard_cb]
    )

    # Exporting the model with default serving function.
    tf.saved_model.save(model, model_export_path)
    return history


Overwriting ./taxifare/trainer/model.py


### Modify code to read data from and write checkpoint files to GCS 

If you look closely above, you'll notice a new function, `train_and_evaluate` that wraps the code that actually trains the model. This allows us to parametrize the training by passing a dictionary of parameters to this function (e.g, `batch_size`, `num_examples_to_train_on`, `train_data_path` etc.)

This is useful because the output directory, data paths and number of train steps will be different depending on whether we're training locally or in the cloud. Parametrizing allows us to use the same code for both.

We specify these parameters at run time via the command line. Which means we need to add code to parse command line parameters and invoke `train_and_evaluate()` with those params. This is the job of the `task.py` file. 

In [14]:
%%writefile taxifare/trainer/task.py
# The argparse module makes it easy to write user-friendly command-line interfaces. It parses the defined arguments from the `sys.argv`.
# The argparse module also automatically generates help & usage messages and issues errors when users give the program invalid arguments.
import argparse

from trainer import model


# Write an `task.py` file for adding code to parse command line parameters and invoke `train_and_evaluate()` with those parameters. 
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--batch_size",
        help="Batch size for training steps",
        type=int,
        default=32
    )
    parser.add_argument(
        "--eval_data_path",
        help="GCS location pattern of eval files",
        required=True
    )
    parser.add_argument(
        "--nnsize",
        help="Hidden layer sizes (provide space-separated sizes)",
        nargs="+",
        type=int,
        default=[32, 8]
    )
    parser.add_argument(
        "--nbuckets",
        help="Number of buckets to divide lat and lon with",
        type=int,
        default=10
    )
    parser.add_argument(
        "--lr",
        help = "learning rate for optimizer",
        type = float,
        default = 0.001
    )
    parser.add_argument(
        "--num_evals",
        help="Number of times to evaluate model on eval data training.",
        type=int,
        default=5
    )
    parser.add_argument(
        "--num_examples_to_train_on",
        help="Number of examples to train on.",
        type=int,
        default=100
    )
    parser.add_argument(
        "--output_dir",
        help="GCS location to write checkpoints and export models",
        required=True
    )
    parser.add_argument(
        "--train_data_path",
        help="GCS location pattern of train files containing eval URLs",
        required=True
    )
    parser.add_argument(
        "--job-dir",
        help="this model ignores this field, but it is required by gcloud",
        default="junk"
    )
    args = parser.parse_args()
    hparams = args.__dict__
    hparams.pop("job-dir", None)

    model.train_and_evaluate(hparams)


Overwriting taxifare/trainer/task.py


### Run trainer module package locally

Now we can test our training code locally as follows using the local test data. We'll run a very small training job over a single file with a small batch size and one eval step.

In [15]:
%%bash

# Testing our training code locally
EVAL_DATA_PATH=./taxifare/tests/data/taxi-valid*
TRAIN_DATA_PATH=./taxifare/tests/data/taxi-train*
OUTPUT_DIR=./taxifare-model

test ${OUTPUT_DIR} && rm -rf ${OUTPUT_DIR}
export PYTHONPATH=${PYTHONPATH}:${PWD}/taxifare
    
python3 -m trainer.task \
--eval_data_path $EVAL_DATA_PATH \
--output_dir $OUTPUT_DIR \
--train_data_path $TRAIN_DATA_PATH \
--batch_size 5 \
--num_examples_to_train_on 100 \
--num_evals 1 \
--nbuckets 10 \
--lr 0.001 \
--nnsize 32 8

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dropoff_latitude (InputLayer)   [(None,)]            0                                            
__________________________________________________________________________________________________
dropoff_longitude (InputLayer)  [(None,)]            0                                            
__________________________________________________________________________________________________
pickup_longitude (InputLayer)   [(None,)]            0                                            
__________________________________________________________________________________________________
pickup_latitude (InputLayer)    [(None,)]            0                                            
______________________________________________________________________________________________

2021-01-27 17:54:03.088382: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-27 17:54:03.089978: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2021-01-27 17:54:03.389035: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2021-01-27 17:54:03.389099: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2021-01-27 17:54:03.389139: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2021-01-27 17:54:03.411555: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-01-27 17:54:03.412294: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2200145000 Hz
2021-01-27 17:54:05.067942: I tensorflow/core/profil

### Run your training package on Cloud AI Platform

Once the code works in standalone mode locally, you can run it on Cloud AI Platform. To submit to the Cloud we use [`gcloud ai-platform jobs submit training [jobname]`](https://cloud.google.com/sdk/gcloud/reference/ml-engine/jobs/submit/training) and simply specify some additional parameters for AI Platform Training Service:
- jobid: A unique identifier for the Cloud job. We usually append system time to ensure uniqueness
- region: Cloud region to train in. See [here](https://cloud.google.com/ml-engine/docs/tensorflow/regions) for supported AI Platform Training Service regions

The arguments before `-- \` are for AI Platform Training Service.
The arguments after `-- \` are sent to our `task.py`.

Because this is on the entire dataset, it will take a while. You can monitor the job from the GCP console in the Cloud AI Platform section.

In [16]:
%%bash

# Output directory and jobID
OUTDIR=gs://${BUCKET}/taxifare/trained_model_$(date -u +%y%m%d_%H%M%S)
JOBID=taxifare_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBID}
gsutil -m rm -rf ${OUTDIR}

# Model and training hyperparameters
BATCH_SIZE=50
NUM_EXAMPLES_TO_TRAIN_ON=100
NUM_EVALS=100
NBUCKETS=10
LR=0.001
NNSIZE="32 8"

# GCS paths
GCS_PROJECT_PATH=gs://$BUCKET/taxifare
DATA_PATH=$GCS_PROJECT_PATH/data
TRAIN_DATA_PATH=$DATA_PATH/taxi-train*
EVAL_DATA_PATH=$DATA_PATH/taxi-valid*

#TODO 2
gcloud ai-platform jobs submit training $JOBID \
    --module-name=trainer.task \
    --package-path=taxifare/trainer \
    --staging-bucket=gs://${BUCKET} \
    --python-version=3.7 \
    --runtime-version=${TFVERSION} \
    --region=${REGION} \
    -- \
    --eval_data_path $EVAL_DATA_PATH \
    --output_dir $OUTDIR \
    --train_data_path $TRAIN_DATA_PATH \
    --batch_size $BATCH_SIZE \
    --num_examples_to_train_on $NUM_EXAMPLES_TO_TRAIN_ON \
    --num_evals $NUM_EVALS \
    --nbuckets $NBUCKETS \
    --lr $LR \
    --nnsize $NNSIZE 

gs://qwiklabs-gcp-01-40af11de1e73/taxifare/trained_model_210127_175410 us-east1 taxifare_210127_175410


CommandException: 1 files/objects could not be removed.
API [ml.googleapis.com] not enabled on project [689077447228]. Would 
you like to enable and retry (this will take a few minutes)? (y/N)?  
ERROR: (gcloud.ai-platform.jobs.submit.training) User [689077447228-compute@developer.gserviceaccount.com] does not have permission to access projects instance [qwiklabs-gcp-01-40af11de1e73] (or it may not exist): AI Platform Training & Prediction API has not been used in project 689077447228 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/ml.googleapis.com/overview?project=689077447228 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.
- '@type': type.googleapis.com/google.rpc.Help
  links:
  - description: Google developers console API activation
    url: https://console.developers.google.com/apis/api/ml.googleapis.com/overview?project=689077447228
- '@type': type.googleapis.com

CalledProcessError: Command 'b'\n# Output directory and jobID\nOUTDIR=gs://${BUCKET}/taxifare/trained_model_$(date -u +%y%m%d_%H%M%S)\nJOBID=taxifare_$(date -u +%y%m%d_%H%M%S)\necho ${OUTDIR} ${REGION} ${JOBID}\ngsutil -m rm -rf ${OUTDIR}\n\n# Model and training hyperparameters\nBATCH_SIZE=50\nNUM_EXAMPLES_TO_TRAIN_ON=100\nNUM_EVALS=100\nNBUCKETS=10\nLR=0.001\nNNSIZE="32 8"\n\n# GCS paths\nGCS_PROJECT_PATH=gs://$BUCKET/taxifare\nDATA_PATH=$GCS_PROJECT_PATH/data\nTRAIN_DATA_PATH=$DATA_PATH/taxi-train*\nEVAL_DATA_PATH=$DATA_PATH/taxi-valid*\n\n#TODO 2\ngcloud ai-platform jobs submit training $JOBID \\\n    --module-name=trainer.task \\\n    --package-path=taxifare/trainer \\\n    --staging-bucket=gs://${BUCKET} \\\n    --python-version=3.7 \\\n    --runtime-version=${TFVERSION} \\\n    --region=${REGION} \\\n    -- \\\n    --eval_data_path $EVAL_DATA_PATH \\\n    --output_dir $OUTDIR \\\n    --train_data_path $TRAIN_DATA_PATH \\\n    --batch_size $BATCH_SIZE \\\n    --num_examples_to_tra

### (Optional) Run your training package using Docker container

AI Platform Training also supports training in custom containers, allowing users to bring their own Docker containers with any pre-installed ML framework or algorithm to run on AI Platform Training. 

In this last section, we'll see how to submit a Cloud training job using a customized Docker image. 

Containerizing our `./taxifare/trainer` package involves 3 steps:

* Writing a Dockerfile in `./taxifare`
* Building the Docker image
* Pushing it to the Google Cloud container registry in our GCP project

The `Dockerfile` specifies
1. How the container needs to be provisioned so that all the dependencies in our code are satisfied
2. Where to copy our trainer Package in the container and how to install it (`pip install /trainer`)
3. What command to run when the container is ran (the `ENTRYPOINT` line)

In [17]:
%%writefile ./taxifare/Dockerfile
# Writing the Dockerfile
FROM gcr.io/deeplearning-platform-release/tf2-cpu
# TODO 3

COPY . /code

WORKDIR /code

ENTRYPOINT ["python3", "-m", "trainer.task"]

Overwriting ./taxifare/Dockerfile


In [18]:
!gcloud auth configure-docker


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud"
  }
}
Adding credentials for all GCR repositories.
gcloud credential helpers already registered correctly.


In [19]:
%%bash 

# Pushing the docker image to Google Cloud container registry in our GCP project
PROJECT_DIR=$(cd ./taxifare && pwd)
PROJECT_ID=$(gcloud config list project --format "value(core.project)")
IMAGE_NAME=taxifare_training_container
DOCKERFILE=$PROJECT_DIR/Dockerfile
IMAGE_URI=gcr.io/$PROJECT_ID/$IMAGE_NAME

docker build $PROJECT_DIR -f $DOCKERFILE -t $IMAGE_URI

docker push $IMAGE_URI

Sending build context to Docker daemon    126kB
Step 1/4 : FROM gcr.io/deeplearning-platform-release/tf2-cpu
latest: Pulling from deeplearning-platform-release/tf2-cpu
d519e2592276: Pulling fs layer
d22d2dfcfa9c: Pulling fs layer
b3afe92c540b: Pulling fs layer
78332ee3cdc2: Pulling fs layer
934e42656764: Pulling fs layer
e23f9a2a918d: Pulling fs layer
2dacc14920ea: Pulling fs layer
b2f53f1965c6: Pulling fs layer
1c1b599df9e1: Pulling fs layer
7fc4c91bd0ba: Pulling fs layer
ccd026bc5834: Pulling fs layer
5421bac9d5e8: Pulling fs layer
b6ef283f70ef: Pulling fs layer
56e759690c02: Pulling fs layer
8b0f8341eedc: Pulling fs layer
0a452f0b3b77: Pulling fs layer
f26f43ca24db: Pulling fs layer
d6ce83df19cb: Pulling fs layer
8c5b46441b0c: Pulling fs layer
ccd026bc5834: Waiting
5421bac9d5e8: Waiting
b6ef283f70ef: Waiting
56e759690c02: Waiting
8b0f8341eedc: Waiting
0a452f0b3b77: Waiting
78332ee3cdc2: Waiting
f26f43ca24db: Waiting
934e42656764: Waiting
d6ce83df19cb: Waiting
e23f9a2a918d: Waiting
8

**Remark:** If you prefer to build the container image from the command line, we have written a script for that `./taxifare/scripts/build.sh`. This script reads its configuration from the file `./taxifare/scripts/env.sh`. You can configure these arguments the way you want in that file. You can also simply type `make build` from within `./taxifare` to build the image (which will invoke the build script). Similarly, we wrote the script `./taxifare/scripts/push.sh` to push the Docker image, which you can also trigger by typing `make push` from within `./taxifare`.

### Train using a custom container on AI Platform

To submit to the Cloud we use [`gcloud ai-platform jobs submit training [jobname]`](https://cloud.google.com/sdk/gcloud/reference/ml-engine/jobs/submit/training) and simply specify some additional parameters for AI Platform Training Service:
- jobname: A unique identifier for the Cloud job. We usually append system time to ensure uniqueness
- master-image-uri: The uri of the Docker image we pushed in the Google Cloud registry
- region: Cloud region to train in. See [here](https://cloud.google.com/ml-engine/docs/tensorflow/regions) for supported AI Platform Training Service regions


The arguments before `-- \` are for AI Platform Training Service.
The arguments after `-- \` are sent to our `task.py`.

You can track your job and view logs using [cloud console](https://console.cloud.google.com/mlengine/jobs).

In [20]:
%%bash

PROJECT_ID=$(gcloud config list project --format "value(core.project)")
BUCKET=$PROJECT_ID
REGION="us-central1"

# Output directory and jobID
OUTDIR=gs://${BUCKET}/taxifare/trained_model
JOBID=taxifare_container_$(date -u +%y%m%d_%H%M%S)
echo ${OUTDIR} ${REGION} ${JOBID}
gsutil -m rm -rf ${OUTDIR}

# Model and training hyperparameters
BATCH_SIZE=50
NUM_EXAMPLES_TO_TRAIN_ON=100
NUM_EVALS=100
NBUCKETS=10
NNSIZE="32 8"

# AI-Platform machines to use for training
MACHINE_TYPE=n1-standard-4
SCALE_TIER=CUSTOM

# GCS paths.
GCS_PROJECT_PATH=gs://$BUCKET/taxifare
DATA_PATH=$GCS_PROJECT_PATH/data
TRAIN_DATA_PATH=$DATA_PATH/taxi-train*
EVAL_DATA_PATH=$DATA_PATH/taxi-valid*

IMAGE_NAME=taxifare_training_container
IMAGE_URI=gcr.io/$PROJECT_ID/$IMAGE_NAME

gcloud beta ai-platform jobs submit training $JOBID \
   --staging-bucket=gs://$BUCKET \
   --region=$REGION \
   --master-image-uri=$IMAGE_URI \
   --master-machine-type=$MACHINE_TYPE \
   --scale-tier=$SCALE_TIER \
  -- \
  --eval_data_path $EVAL_DATA_PATH \
  --output_dir $OUTDIR \
  --train_data_path $TRAIN_DATA_PATH \
  --batch_size $BATCH_SIZE \
  --num_examples_to_train_on $NUM_EXAMPLES_TO_TRAIN_ON \
  --num_evals $NUM_EVALS \
  --nbuckets $NBUCKETS \
  --nnsize $NNSIZE 


gs://qwiklabs-gcp-01-40af11de1e73/taxifare/trained_model us-central1 taxifare_container_210127_175644


CommandException: 1 files/objects could not be removed.
API [ml.googleapis.com] not enabled on project [689077447228]. Would 
you like to enable and retry (this will take a few minutes)? (y/N)?  
ERROR: (gcloud.beta.ai-platform.jobs.submit.training) User [689077447228-compute@developer.gserviceaccount.com] does not have permission to access projects instance [qwiklabs-gcp-01-40af11de1e73] (or it may not exist): AI Platform Training & Prediction API has not been used in project 689077447228 before or it is disabled. Enable it by visiting https://console.developers.google.com/apis/api/ml.googleapis.com/overview?project=689077447228 then retry. If you enabled this API recently, wait a few minutes for the action to propagate to our systems and retry.
- '@type': type.googleapis.com/google.rpc.Help
  links:
  - description: Google developers console API activation
    url: https://console.developers.google.com/apis/api/ml.googleapis.com/overview?project=689077447228
- '@type': type.googleapi

CalledProcessError: Command 'b'\nPROJECT_ID=$(gcloud config list project --format "value(core.project)")\nBUCKET=$PROJECT_ID\nREGION="us-central1"\n\n# Output directory and jobID\nOUTDIR=gs://${BUCKET}/taxifare/trained_model\nJOBID=taxifare_container_$(date -u +%y%m%d_%H%M%S)\necho ${OUTDIR} ${REGION} ${JOBID}\ngsutil -m rm -rf ${OUTDIR}\n\n# Model and training hyperparameters\nBATCH_SIZE=50\nNUM_EXAMPLES_TO_TRAIN_ON=100\nNUM_EVALS=100\nNBUCKETS=10\nNNSIZE="32 8"\n\n# AI-Platform machines to use for training\nMACHINE_TYPE=n1-standard-4\nSCALE_TIER=CUSTOM\n\n# GCS paths.\nGCS_PROJECT_PATH=gs://$BUCKET/taxifare\nDATA_PATH=$GCS_PROJECT_PATH/data\nTRAIN_DATA_PATH=$DATA_PATH/taxi-train*\nEVAL_DATA_PATH=$DATA_PATH/taxi-valid*\n\nIMAGE_NAME=taxifare_training_container\nIMAGE_URI=gcr.io/$PROJECT_ID/$IMAGE_NAME\n\ngcloud beta ai-platform jobs submit training $JOBID \\\n   --staging-bucket=gs://$BUCKET \\\n   --region=$REGION \\\n   --master-image-uri=$IMAGE_URI \\\n   --master-machine-type=$MAC

**Remark:** If you prefer submitting your jobs for training on the AI-platform using the command line, we have written the `./taxifare/scripts/submit.sh` for you (that you can also invoke using `make submit` from within `./taxifare`). As the other scripts, it reads it configuration variables from `./taxifare/scripts/env.sh`.

Copyright 2020 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License